In [1]:
from keras.models  import load_model
import keras.backend as K

import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer

import pandas as pd
import numpy as np

from gcnn import GCNN
aspects = ['population', 'intervention', 'outcome']

Using TensorFlow backend.


[]


In [3]:
vec = cPickle.load(open('../../preprocess/allfields_with_embedding_19995.p', 'rb'))

In [3]:
index = vec.index
C = {'inputs' : ['abstract', 'population', 'intervention', 'outcome']}
for input in C['inputs'] :
    input_range = index[input]
    C[input] = vec.X[input_range[0]:input_range[1]]

In [4]:
model = load_model('../store/weights/single_setup/ThuDec1414:54:212017GatedCNN19K/loss.h5', 
                   custom_objects={'contrastive_loss' : lambda a,b : a, '<lambda>' : lambda a, b : a})
#results_folder = '../store/results/single_message_ThuOct1919:10:492017/'

In [5]:
model_c1 = {}
for aspect in aspects :
    model_c1[aspect] = K.function(model.get_layer('pool_'+aspect[0].upper()).inputs, 
                          [model.get_layer('pool_'+aspect[0].upper()).layers[16].output])

In [9]:
pool_cdsr = {}
X_abs = C['abstract']
from sklearn.preprocessing import normalize


for aspect in aspects :
    pool_cdsr[aspect] = []
    bs = 1000
    for i in range(int(np.floor(len(X_abs)/bs)) + 1) :
        val = model_c1[aspect]([X_abs[i*bs:(i+1)*bs]])[0][:, :, 0]
        val = np.dstack((X_abs[i*bs:(i+1)*bs], val))
        pool_cdsr[aspect].append(val)
    pool_cdsr[aspect] = np.concatenate(pool_cdsr[aspect])

filter_aspect = {}
for aspect in aspects :
    filter_aspect[aspect] = pool_cdsr[aspect]

In [7]:
activated_words = {}
for aspect in aspects :
    filters = filters_pico[(aspect, 'abstract')]
    word_activations = np.zeros((vec.vocab_size, filters.shape[2]))
    for i in range(filters.shape[2]) :
        filter_i = filters[:,:,i]
        word2filter = np.dstack((C['abstract'][sample], filter_i))
        filter_w2f = word2filter[word2filter[:,:,1] > 0.0]
        for word in filter_w2f :
            word_activations[int(word[0]), i] = word[1]
    activated_words[aspect] = word_activations

In [8]:
top_words_df = pd.DataFrame()
top_words_size = 10
for aspect in aspects :
    popwords = activated_words[aspect].sum(axis=1)
    sorted_idx = np.argsort(popwords)
    top_words = sorted_idx[-top_words_size:]
    top_words = pd.Series([v for k, v in vec.idx2word.items() if k in top_words])
    top_words_df[aspect] = top_words
top_words_df

,population,intervention,outcome
0,young,young,young
1,younger,younger,younger
2,youth,youth,youth
3,youths,youths,youths
4,yr,yr,yr
5,yrs,yrs,yrs
6,z,z,z
7,zero,zero,zero
8,zidovudine,zidovudine,zidovudine
9,zinc,zinc,zinc


<b>Evaluating Filters on Cohen Data</b>

In [20]:
df = pd.read_csv('../data/files/test_cohen_dedup.csv')
cohenvec = cPickle.load(open('../../preprocess/cohendata_dedup_19995.p', 'rb'))
cohenvec = cPickle.load(open('../../preprocess/decision_aids_vec_19995.p', 'rb'))
#cohenvec =cPickle.load(open('../../pubmed/pubmed_data_vec_5000_dedup_n.p'))

In [21]:
pool_cdsr = {}
X_abs = cohenvec.X
from sklearn.preprocessing import normalize


for aspect in aspects :
    pool_cdsr[aspect] = []
    bs = 100
    for i in range(int(np.floor(len(X_abs)/bs)) + 1) :
        val = model_c1[aspect]([X_abs[i*bs:(i+1)*bs]])[0][:, :, 0]
        val = np.dstack((X_abs[i*bs:(i+1)*bs], val))
        pool_cdsr[aspect].append(val)
    pool_cdsr[aspect] = np.concatenate(pool_cdsr[aspect])
    #pool_cdsr[aspect] = softmax(pool_cdsr[aspect][:,:,0], axis=1)[:,:,np.newaxis]

filter_aspect = {}
for aspect in aspects :
    filter_aspect[aspect] = pool_cdsr[aspect]

In [22]:
activated_words = {}
for aspect in aspects :
    filters = filter_aspect[aspect]
    word_activations = np.zeros((vec.vocab_size, 1))
    word2filter = filters.reshape(-1, filters.shape[-1])
    for word in word2filter :
        word_activations[int(word[0]), 0] = word[1]
    activated_words[aspect] = word_activations
    
joint = np.concatenate([activated_words[aspect] for aspect in aspects], axis=1)
joint = joint[2:, :]

In [25]:
top_words_df = pd.DataFrame()
top_words_size = 10
for i, aspect in enumerate(aspects) :
    popwords = joint[:, i]
    sorted_idx = np.argsort(popwords)
    top_words = sorted_idx[-top_words_size:]
    print [str(v) for k, v in vec.idx2word.items() if k in top_words]
    top_words = pd.Series([v for k, v in vec.idx2word.items() if k in top_words])
    
    top_words_df[aspect] = top_words
top_words_df

['alternatives', 'color', 'curosurf', 'delivering', 'imv', 'inotropic', 'malawian', 'presenting', 'staffordshire', 'ui']
['aides', 'alternatives', 'base', 'effluent', 'elicitation', 'evacuation', 'guided', 'intervened', 'practicality', 'ward']
['aides', 'atra', 'disorganized', 'drs', 'elicitation', 'fibreoptic', 'glycated', 'meniscectomy', 'verruca', 'weekly']


,population,intervention,outcome
0,alternatives,aides,aides
1,color,alternatives,atra
2,curosurf,base,disorganized
3,delivering,effluent,drs
4,imv,elicitation,elicitation
5,inotropic,evacuation,fibreoptic
6,malawian,guided,glycated
7,presenting,intervened,meniscectomy
8,staffordshire,practicality,verruca
9,ui,ward,weekly


In [26]:
from colors import color

In [29]:
from random import randint
for m in range(30):
    s = randint(0, len(filter_aspect[aspects[0]])-1)
    gates = {}
    for aspect in aspects :
        gate0 = filter_aspect[aspect][s, :, 1]
        nz = sum([1 for i in filter_aspect[aspect][s, :, 0] if i == 0])
        
        gate0 = np.convolve(gate0, [1./3 for _ in range(3)], mode='same')
        gate0 = (gate0 - gate0[nz:].mean(axis=0))/ gate0[nz:].std(axis=0)
        gate0 = (gate0 - gate0[nz:].min(axis=0)) / (gate0[nz:].max(axis=0) - gate0[nz:].min(axis=0))
        gate0 = gate0.clip(0, 1)


        gates[aspect] = gate0


    X_words = [vec.idx2word[int(i)] for i in filter_aspect[aspect][s, :, 0]]
    for v, aspect in enumerate(aspects) :
        for i, j in zip(X_words, gates[aspect]) :
            c = [255, 255, 255]
            for k in range(3) :
                if k != v :
                    c[k] = int((1 - j)*255)
            if i != '[0]' :
                print color(i, bg=c),
        print ""
    print "--------------------------------------------------------------------------------------"+str(m)

in response to the isolation of the qqq gene , a breast - ovarian cancer - susceptibility gene , unk companies are already marketing genetic tests to health care providers and to the public . initial studies indicate interest in qqq testing in the general public and in populations at high risk . however , the optimal strategies for educating and counseling individuals have yet to be determined . 
in response to the isolation of the qqq gene , a breast - ovarian cancer - susceptibility gene , unk companies are already marketing genetic tests to health care providers and to the public . initial studies indicate interest in qqq testing in the general public and in populations at high risk . however , the optimal strategies for educating and counseling individuals have yet to be determined . 
in response to the isolation of the qqq gene , a breast - ovarian cancer - susceptibility gene , unk companies are already marketing genetic tests to health care providers and to the public . initial 

the efficacy of prostate cancer screening is uncertain , and professional organizations recommend educating patients about potential harms and benefits . we evaluated the effect of a videotape decision aid on promoting informed decision making about prostate cancer screening among primary care patients . 
the efficacy of prostate cancer screening is uncertain , and professional organizations recommend educating patients about potential harms and benefits . we evaluated the effect of a videotape decision aid on promoting informed decision making about prostate cancer screening among primary care patients . 
--------------------------------------------------------------------------------------12
use of the prostate - specific antigen ( psa ) as a screening test remains highly controversial , particularly in older men . this study was undertaken to assess the impact of information on the preferences of older men for such screening . 
use of the prostate - specific antigen ( psa ) as a scr

in response to the isolation of the qqq gene , a breast - ovarian cancer - susceptibility gene , unk companies are already marketing genetic tests to health care providers and to the public . initial studies indicate interest in qqq testing in the general public and in populations at high risk . however , the optimal strategies for educating and counseling individuals have yet to be determined . 
--------------------------------------------------------------------------------------22
previous studies have not assessed whether evidence - based information about the outcomes of colorectal cancer screening increases informed choice among people from a range of socioeconomic backgrounds nor have they assessed whether this can be administered away from a health - care provider . 
previous studies have not assessed whether evidence - based information about the outcomes of colorectal cancer screening increases informed choice among people from a range of socioeconomic backgrounds nor have th